In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
# chattem:template을 message로부터 만듬 / prompttem:string을 이용해 template만듬

chat = ChatOpenAI(temperature=0.1)
# b = chat.predict("How many planets are there?")

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

In [ ]:
# 직접 message를 지정해서 실행. 사용할 라이브러리 다 불러와야 함.

from langchain.schema import HumanMessage, AIMessage, SystemMessage
# System:역할지정/ AI:ai 시작할 때 할말/ Human:물어볼 거.

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, What is your name?")
]

chat.predict_messages(messages)

In [ ]:
# chatprompttemplate 사용해서 더 편하게 template 작성
# 커스텀 하기 더 쉬어짐.
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}"),
        ("human", "What is the distance between {country_a} and {country_b}. Also, What is your name?")
    ]
)

prompt = template.from_messages(
    language="Greak",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)